In [14]:
from kafka import KafkaConsumer
import time
import json
from pymongo import MongoClient
import pandas as pd
import os
import csv
client = MongoClient('mongodb://localhost:27017/')
db = client['tp4']
collection = db['t_3']

In [15]:
import csv
import pymongo

# Tạo kết nối đến MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['tp4']
collection = db['t_4']

# Đọc dữ liệu từ tệp CSV và chuyển đổi thành từ điển
def read_csv_to_dict(filename):
    data_list = []
    with open(filename, mode='r') as file:  # Sửa dòng này
        reader = csv.DictReader(file)
        for row in reader:
            data_list.append(row)
    return data_list

# Chèn dữ liệu vào MongoDB
def insert_data_to_mongo(data_list):
    for i in data_list:  # Kiểm tra xem danh sách dữ liệu có rỗng không
        collection.insert_one(i)
        print("ok")

# Đọc dữ liệu từ tệp CSV và chèn vào MongoDB
csv_file = "test_data_comparison.csv"
data_list = read_csv_to_dict(csv_file)
insert_data_to_mongo(data_list)


ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
o

In [16]:
import csv
import pymongo
import time
import os

# Tạo kết nối đến MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['tp4']
collection = db['t_4']

# Đọc dữ liệu từ tệp CSV và chuyển đổi thành từ điển
def read_csv_to_dict(filename):
    data_list = []
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data_list.append(row)
    return data_list

# Chèn dữ liệu vào MongoDB
def insert_data_to_mongo(data_list):
    for i in data_list:
        print("Data to insert:", i)  # Thêm dòng này để kiểm tra dữ liệu
        collection.insert_one(i)
        print("Inserted:", i)

# Hàm theo dõi sự thay đổi của tệp CSV
def watch_file(filename, interval=10):
    last_mtime = None
    while True:
        try:
            current_mtime = os.path.getmtime(filename)
            print(f"Current mtime: {current_mtime}, Last mtime: {last_mtime}")  # In thông tin mtime
            if last_mtime is None:
                last_mtime = current_mtime

            if current_mtime > last_mtime:
                print(f"Detected change in {filename}. Reading new data...")
                data_list = read_csv_to_dict(filename)
                insert_data_to_mongo(data_list)
                last_mtime = current_mtime
            time.sleep(interval)
        except KeyboardInterrupt:
            print("Stopping file watch.")
            break

# Theo dõi tệp CSV và đẩy dữ liệu mới lên MongoDB
csv_file = "test_data_comparison.csv"
watch_file(csv_file, interval=10)


Current mtime: 1724257457.876366, Last mtime: None
Current mtime: 1724257457.876366, Last mtime: 1724257457.876366
Stopping file watch.


In [ ]:

# create Kafka consumer instance
consumer = KafkaConsumer("sic-kafka-2", bootstrap_servers='localhost:9092', auto_offset_reset='earliest', enable_auto_commit=False)

timeout_duration = 20  # thời gian chờ (20s)
last_received_time = time.time()

while True:
    records = consumer.poll(timeout_ms=1000)  # poll tin nhắn với timeout 1 giây
    if records:
        last_received_time = time.time()  # cập nhật thời gian khi nhận được tin nhắn mới
        for message_batch in records.values():
            for msg in message_batch:
                print(msg.value.decode())
                 # Convert message to JSON if it is not already
                decoded_message = msg.value.decode()
                try:
                    data = json.loads(decoded_message)
                except json.JSONDecodeError:
                    data = {"message": decoded_message}  # Nếu không phải JSON, lưu dưới dạng văn bản

                # Insert data into MongoDB
                collection.insert_one(data)
    # else:
    #     # kiểm tra nếu đã quá thời gian chờ mà không có tin nhắn mới
    #     if time.time() - last_received_time >= timeout_duration:
    #         break
